In [5]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import f1_score, confusion_matrix

In [4]:
data = pd.read_csv("learn.csv")

data preprocessing:
* change "insert_time" and "company_enter_month" to datetime format
* one-hot-encode "income_type"
* only take useful columns 

In [6]:
del data ["Unnamed: 0"]
data.insert_time = pd.to_datetime(data.insert_time)
data.company_enter_month = pd.to_datetime(data.company_enter_month)
c1 = pd.get_dummies(data.income_type)
data = pd.concat([data, c1], axis = 1)
del data['income_type']

In [7]:
data = data[["application_id", "loan_limit", "loan_rate", "credit_score", "company_enter_month","insert_time",
                  "desired_amount", "EARNEDINCOME", "is_applied"]]

Problem 1: <br>
Predict which applications will be applied<br>
divide test & train data based on date

In [143]:
test_data = data[data['insert_time']>='2022-05-16']
train_data = data[data['insert_time']<'2022-05-16']
test_data_grouped = test_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)
train_data_grouped = train_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)
test_label_grouped = test_data_grouped["is_applied"]
del test_data_grouped["is_applied"]
train_label_grouped = train_data_grouped["is_applied"]
del train_data_grouped["is_applied"]

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_6166/1987880793.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.reset_index except for the argument 'level' will be keyword-only.
  test_data_grouped = test_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)
/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_6166/1987880793.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.reset_index except for the argument 'level' will be keyword-only.
  train_data_grouped = train_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)


Use CatBoost for Problem 1

In [144]:
by_application = data.groupby("application_id").max()
rate = by_application.is_applied.mean()

In [145]:
model = CatBoostClassifier(iterations=2,
                           depth=15,
                           learning_rate=0.89,
                           loss_function='Logloss')
train = Pool(
    data = train_data_grouped,
    label = train_label_grouped,
    weight = train_label * (1-2*rate) + rate
)
model.fit(train)
predict = model.predict(test_data_grouped)
f1_score(test_label_grouped, predict)

0:	learn: 0.6562759	total: 563ms	remaining: 563ms
1:	learn: 0.6503298	total: 987ms	remaining: 0us


0.6605963644024834

Problem 2: <br>
Predict which products will be chosen among the chosen applications
* Take applications which are predicted to be applied by Problem 1
* Assume people apply for loans that have the least loan_rate, until they have fulfilled their desired_amount

In [146]:
test_data_grouped.loc[:, "predict"] = predict
applied_application_ids = test_data_grouped.loc[test_data_grouped.predict == 1, "application_id"]
applied_applications = data.loc[data.application_id.isin(applied_application_ids)]
applied_applications = applied_applications[["application_id", "loan_limit", "loan_rate", "is_applied", "desired_amount"]]
applied_applications = applied_applications.sort_values(by = ["application_id", "loan_rate"])

In [151]:
applied_applications.loc[:, ["sum", "is_enough", "predict"]] = 0
applied_applications.iloc[0, 5] = applied_applications.iloc[0, 1]

In [152]:
for i in range (1, len(applied_applications)):
    applied_applications.iloc[i, 5] = applied_applications.iloc[i, 1]
    if (applied_applications.iloc[i, 0] == applied_applications.iloc[i-1, 0]):
        applied_applications.iloc[i, 5] += applied_applications.iloc[i-1, 5]

In [153]:
applied_applications["is_enough"] = applied_applications["sum"] > applied_applications["desired_amount"]

In [157]:
applied_applications.iloc[0, 7] = 1
applied_applications.iloc[1, 7] = 1

In [158]:
for i in range(2, len(applied_applications)):
    if (applied_applications.iloc[i, 0] == applied_applications.iloc[i-1, 0]):
        if (not applied_applications.iloc[i, 6]):
            applied_applications.iloc[i, 7] = 1
        elif (not applied_applications.iloc[i-1, 6]):
            applied_applications.iloc[i, 7] = 1
        elif (not applied_applications.iloc[i-2, 6]):
            applied_applications.iloc[i, 7] = 1
    else:
        applied_applications.iloc[i, 7] = 1

In [164]:
f1_score(applied_applications.is_applied, applied_applications.predict)

0.4297124676722598

Use the Results of Problem 1 & 2 to Predict the Total Problem

In [159]:
applied_products_id = applied_applications.loc[applied_applications.predict == 1].index

In [161]:
test_data.loc[:, "predict"] = 0
test_data.loc[applied_products_id, "predict"] = 1

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_6166/912087388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[:, "predict"] = 0


In [165]:
f1_score(test_data.is_applied, test_data.predict)

0.3733890659272441

In [167]:
confusion_matrix(test_data.is_applied, test_data.predict).ravel()

array([1672209,   89701,   61707,   45111])